In [1]:
# visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import brainpy as bp
import brainpy.math as bm
import seaborn as sns
from scipy import stats
import itertools
import pandas as pd
bm.set_platform('cpu')
from scipy.signal import find_peaks

/home/spirit/anaconda3/envs/sbi_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_path = []
file_path.append('data/WT_single1.csv')
file_path.append('data/WT_single2.csv') 

WT_train1 = pd.read_csv(file_path[0])
WT_train2 = pd.read_csv(file_path[1])
single_spike_amp = [62.1948, 61.5235, 63.5986, 64.5752, 64.1174]

In [3]:
train1_time = WT_train1.iloc[:-2, 0].to_numpy()
train1_voltages = WT_train1.iloc[:-2, 2].to_numpy()   

train2_time = WT_train2.iloc[:-2, 0].to_numpy()
train2_voltages = WT_train2.iloc[:-2, 3:].to_numpy()

In [4]:
total_time = 1.0/2* 1000.  # 总时长 (s)
start_time = 0.1156/2 * 1000.  # 刺激启始时刻 (s)
end_time = 0.5156/2 * 1000.  # 刺激结束时刻 (s)


I_current_amplitude   = [30.0 ,25.0, 20.0, 15.0]  # 刺激电流幅值 (nA)
syn_current, duration = bp.inputs.section_input(values=[0, I_current_amplitude, 0.],
                                             durations=[start_time, end_time - start_time, total_time - end_time],
                                             return_length=True,
                                             dt=0.1)

In [5]:
data_wt = np.load("data/spikes.npy")
real_value = data_wt[:,0]
real_time = np.linspace(0, 0.5, len(real_value))
print(data_wt.shape)

(9999, 4)


In [6]:
spikes_exp = []    
isi_exp_list = [ ]
rest_exp_list = [ ]
v = data_wt[:,0]
threshold = 10.
distances = 150.
spike_window_exp = 1600  # Number of points to take around the peak

peaks, _ = find_peaks(v, height= threshold, distance=distances)
isi_exp_list.append(np.diff(real_time[peaks])) 

for peak in peaks:
    if peak - spike_window_exp // 2 >= 0 and peak + spike_window_exp // 2 < len(v):
        spike_segment = v[peak - spike_window_exp // 2 : peak + spike_window_exp // 2]
        spikes_exp.append(spike_segment)

rest_exp_list.append(np.mean(v[real_time>0.4]))

In [7]:
spike_window_exp1 = 400
peaks, _ = find_peaks(train1_voltages, height= 25, distance=distances)
v = train1_voltages
# isi_exp_list.append(np.diff(train1_time[peaks]))  

for peak in peaks:
    if peak - spike_window_exp1 // 2 >= 0 and peak + spike_window_exp1 // 2 < v.shape[0]:
        spike_segment = v[peak - spike_window_exp1 // 2 : peak + spike_window_exp1 // 2]
        spikes_exp.append(spike_segment)

rest_exp_list.append(np.mean(v[train1_time> 3.]))  

In [8]:
for i in range(train2_voltages.shape[1]):
    v = train2_voltages[:,i]
    peaks, _ = find_peaks(v, height=threshold, distance=distances)
    isi_exp = np.diff(train2_time[peaks]) 
    isi_exp_list.append(isi_exp)
    for peak in peaks:
        if peak - spike_window_exp // 2 >= 0 and peak + spike_window_exp // 2 < len(v):
            spike_segment = v[peak - spike_window_exp // 2 : peak + spike_window_exp // 2]
            spikes_exp.append(spike_segment)
    rest_exp_list.append(np.mean(v[train2_time> 0.5]))                    

In [9]:
wt_exp_amp = []
for spike in spikes_exp:
    wt_exp_amp.append(np.max(spike) - np.min(spike))
wt_exp_amp += single_spike_amp
wt_exp_amp = np.array(wt_exp_amp)
print(wt_exp_amp)


[65.5213 59.5398 60.5164 60.4859 59.4482 59.2651 59.6924 61.1878 61.7371
 61.8286 60.9436 61.3708 60.5163 60.0281 59.7534 58.5633 58.2275 55.7556
 55.9387 54.84   52.7648 52.0935 60.1502 61.4624 59.021  60.0281 61.6455
 58.4412 58.5022 60.3942 60.2112 57.3425 58.3191 60.1502 60.6384 57.4341
 58.1359 59.4177 59.7229 59.9975 56.5491 57.9834 59.3566 58.9905 58.8378
 61.5845 62.1948 61.5235 63.5986 64.5752 64.1174]


In [10]:
np.mean(wt_exp_amp)

59.614586274509804

In [11]:
print(isi_exp_list)

[array([0.04670934, 0.04675935, 0.04830966]), array([0.05912]), array([0.05832, 0.05816]), array([0.06004, 0.0592 , 0.16422]), array([0.0583 , 0.05782, 0.05816]), array([0.0578 , 0.056  , 0.05916, 0.06046]), array([0.05912, 0.05998, 0.06216, 0.06282, 0.0632 ])]


In [12]:
isi_exp = np.concatenate(isi_exp_list) 
print(np.mean(isi_exp))

0.06265801693672068


In [13]:
WT_sim_trace = np.load("data/WT_sim.npz")

In [14]:
spikes_sim = []    
spike_window_sim = 800
v = WT_sim_trace['aike'].flatten()
time_sim = WT_sim_trace['aike_time'].flatten()
peaks, _ = find_peaks(v, height= threshold, distance=distances)
isi_sim = np.diff(time_sim[peaks])

for peak in peaks:
    if peak - spike_window_sim // 2 >= 0 and peak + spike_window_sim // 2 < len(v):
        spike_segment = v[peak - spike_window_sim // 2 : peak + spike_window_sim // 2]
        spikes_sim.append(spike_segment)

wt_sim_amp = []
for spike in spikes_sim:
    wt_sim_amp.append(np.max(spike) - np.min(spike))        

In [15]:
print(np.mean(wt_sim_amp))
np.diff(time_sim[peaks])

59.18956


array([0.05191038, 0.04780956, 0.04670934])

In [16]:
isi_exp_list_trial = isi_exp_list
for i in range(1, len(isi_exp_list)):
    isi_exp_list_trial[i] = isi_exp_list[i] - 0.012
isi_exp_list_trial.pop()
print(isi_exp_list_trial)

[array([0.04670934, 0.04675935, 0.04830966]), array([0.04712]), array([0.04632, 0.04616]), array([0.04804, 0.0472 , 0.15222]), array([0.0463 , 0.04582, 0.04616]), array([0.0458 , 0.044  , 0.04716, 0.04846])]


In [17]:
rest_exp_list

[-25.542156600000002,
 -28.119455573505654,
 -29.137604831449433,
 -28.40654050215064,
 -27.23892820846254,
 -27.657455336600982,
 -27.25849304791437,
 -26.08191588476543]

In [18]:
WT_exp_rest = np.array(rest_exp_list)

In [20]:
WT_exp_isi = np.concatenate(isi_exp_list_trial)
v_sim = WT_sim_trace['aike'].flatten()
time_sim = WT_sim_trace['aike_time'].flatten()
WT_sim_rest = np.mean(v_sim[time_sim>0.4])
print(WT_sim_rest)

-23.796795


In [21]:
print(rest_exp_list)

[-25.542156600000002, -28.119455573505654, -29.137604831449433, -28.40654050215064, -27.23892820846254, -27.657455336600982, -27.25849304791437, -26.08191588476543]


In [22]:
np.save("data/WT_exp_amp.npy", wt_exp_amp)
np.save("data/WT_exp_isi.npy", WT_exp_isi)
np.save("data/WT_sim_amp.npy", wt_sim_amp)
np.save("data/WT_sim_isi.npy", isi_sim)
np.save("data/WT_exp_rest.npy", WT_exp_rest)